In [1]:
import os
from experiment.experiment_cv import ExperimentCV
from cornac.data.reader import Reader
from eval_methods.base_method import BaseMethod
from cornac.metrics import Recall,NDCG

In [2]:
from models.base_popular import BasePopular
from models.base_random import BaseRandom
from models.base_cosine import BaseCosine
from models.base_gram import BaseGram
from models.base_jaccard import BaseJaccard
from models.ease import EASE
from models.rp3beta import RP3Beta
from models.recwalk_pr import Recwalk_PR
from models.recwalk_k import Recwalk_K

from models.user_knn_pearson import UserKNN
from models.tars import TARS
from models.acorec import AcoRec

In [3]:
from models.acorec import AcoRec

In [4]:
# "ml1m45" | "pinterest" | "netflix"
dataset_name = "ml1m45" 
# "cs" for cold-start | "lt" for long-tail
scenario = "lt" 

SEED = 12345
cv_fold_size = 5
long_tail = False
if scenario=="lt":
    long_tail = True

# Instantiate evaluation measures
rec_010 = Recall(k=10)
rec_020 = Recall(k=20)
ndcg_010 = NDCG(k=10)
ndcg_020 = NDCG(k=20)
    
reader = Reader()

eval_metric = ndcg_010
result_metrics = [rec_010, rec_020, ndcg_010, ndcg_020]

current_directory = os.getcwd()
DS_FOLDER = os.path.join(current_directory, "datasets/")
eval_methods = []
for fold in range(1,cv_fold_size+1):
    # Prepare datasets for eval.
    train_data = reader.read(DS_FOLDER+"{}_{}_{}.tr".format(scenario,dataset_name,fold))
    test_data = reader.read(DS_FOLDER+"{}_{}_{}.ts".format(scenario,dataset_name,fold))
    print("Fold : {} created".format(fold))
    # Instantiate a Base evaluation method using the provided train and test sets
    eval_methods.append ( BaseMethod.from_splits(
        train_data=train_data, test_data=test_data, val_data=test_data, exclude_unknowns=True, verbose=False
    ) )

Fold : 1 created
Fold : 2 created
Fold : 3 created
Fold : 4 created
Fold : 5 created


In [5]:
# Base Models

# Instantiate the BaseRandom model
base_random = BaseRandom(
    seed=SEED,
)

# Instantiate the BasePopular model
popular = BasePopular(
    seed=SEED,
)

# Instantiate the BaseCosine model
cosine = BaseCosine(
    seed=SEED,
)

# Instantiate the BaseGram model
gram = BaseGram(
    seed=SEED,
)

# Instantiate the BaseJaccard model
jaccard = BaseJaccard(
    seed=SEED,
)

In [6]:
# Instantiate the EASE model
ease = EASE(
    lamb=50,
    seed=SEED,
    verbose=False,
    posB=False
)

# Instantiate the Rp3beta model
rp3beta = RP3Beta(
    alp=.4,
    bet=-.8,
    normalize=False,
    seed=SEED,
    verbose=False
)

# Instantiate the Recwalk_PR model
recwalk_pr_slim = Recwalk_PR(
    eta=.1,
    l1_ratio=1,
    l2_ratio=.1,
    seed=SEED,
    verbose=False,
)

# Instantiate the Recwalk_K model
recwalk_k_slim = Recwalk_K(
    k=3,
    l1_ratio=1,
    l2_ratio=.1,
    seed=SEED,
    verbose=False,
)

# Instantiate the UserKNN model
user_knn_pearson = UserKNN(
    k=10,
    seed=SEED,
    verbose=False,
    name="UserKNN"
)
        
# Instantiate the TARS model
tars = TARS(
    ant_size=50,
    conf_value=.1,
    seed=SEED,
    verbose=False
)

In [7]:
# Our Acorec Models
acorec_gram = AcoRec(
    seed=SEED,
    long_tail=long_tail,
    imodel="gram",
    likelihood="tanh",
    prob_norm="tanh",
    lr=5e-2,
    sig=2,
    drop_out=0.2,
    ant_size=50,
    archive_size=50
)
acorec_cos = AcoRec(
    seed=SEED,
    long_tail=long_tail,
    imodel="cosine",
    likelihood="sigmoid",
    prob_norm="relu",
    lr=5e-2,
    sig=3,
    drop_out=0.5,
    ant_size=100,
    archive_size=50
)
acorec_jac = AcoRec(
    seed=SEED,
    long_tail=long_tail,
    imodel="jaccard",
    likelihood="tanh",
    prob_norm="relu",
    lr=5e-2,
    sig=3,
    drop_out=0.2,
    ant_size=50,
    archive_size=50
)


In [8]:
# models to be evaluated
models = [
    base_random,
    popular,
    cosine,
    gram,
    jaccard,
    user_knn_pearson,
    tars,
    recwalk_pr_slim,
    recwalk_k_slim,
    ease,
    rp3beta,
    acorec_gram,
    acorec_cos,
    acorec_jac,
]

In [9]:
# Put everything together into an experiment and run it
ExperimentCV(
    eval_method=eval_methods,
    models=models,
    metrics=result_metrics,
    user_based=True,
    save_dir=None,
    show_validation=False,
    aco_eval_repeat=3,
).run()

Training model : BaseRandom
Training model : BasePopular
Training model : BaseCosine
Training model : BaseGram
Training model : BaseJaccard
Training model : UserKNN
Training model : TARS
Training model : Recwalkᴾᴿ
Learning takes 6.997 secs.
Learning takes 7.022 secs.
Learning takes 6.942 secs.
Learning takes 7.027 secs.
Learning takes 6.954 secs.
Training model : Recwalkᴷ⁻ˢᵗᵉᵖ
Learning takes 6.942 secs.
Learning takes 6.935 secs.
Learning takes 6.941 secs.
Learning takes 6.977 secs.
Learning takes 6.916 secs.
Training model : EASEᴿ
Training model : RP3Beta
Training model : AcoRec
Using Coordinate Descent! 
Using Coordinate Descent! 
Using Coordinate Descent! 
Using Coordinate Descent! 
Using Coordinate Descent! 
Using Coordinate Descent! 
Using Coordinate Descent! 
Using Coordinate Descent! 
Using Coordinate Descent! 
Using Coordinate Descent! 
Training model : AcoRec
Training model : AcoRec

 ALL FOLD AVERAGE TEST:
...
                 | NDCG@10 | NDCG@20 | Recall@10 | Recall@20 | Tra